In [2]:
import torch #pytorch library that helps in building the deep leanring algorithms
from torch import nn ##nn means neural network
from torch.utils.data import DataLoader #performs the process of batching
from torchvision import datasets #Inbuild dataset that pytorch library has - FashionMnist Dataset
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

100%|██████████| 26421880/26421880 [00:06<00:00, 3894316.33it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 199757.45it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3740604.97it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6016237.67it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

**Batching of this data**





In [6]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [7]:

for X,y in test_dataloader: #Image - Color image shape (batch_size,number of channel,length,width)
  print(X.shape)            #Image - Black and white image - number of channels is 1
  print(y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [8]:

device = "cuda" if torch.cuda.is_available() else "cpu" #torch.cuda.is_available() checks for your system has gpu or cpu
device

'cuda'

In [9]:
class NeuralNetwork(nn.Module): #Child class and nn.Module is a parent class -- (defined in the pytorch library)

  def __init__(self): #declare the architecture
    super().__init__() #basically initailizes all the variables of the parent class
    self.flatten = nn.Flatten() #28x28 image into a 764x1 vector
    self.linear1 = nn.Linear(28*28,512)
    self.linear2 = nn.Linear(512,512)
    self.linear3 = nn.Linear(512,10)
    self.relu = nn.ReLU()
  def forward(self,x):#is always used to pass the inputs to the neural network
    x = self.flatten(x)
    x = self.linear1(x)
    x = self.relu(x)
    x = self.linear2(x)
    x = self.relu(x)
    x = self.linear3(x)
    return x

In [10]:
model = NeuralNetwork()
model = model.to(device) #copies your entire architecture to the GPU

In [11]:
loss_fn = nn.CrossEntropyLoss() #cross entropy loss
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3) #stochastic Gradient descent

In [12]:
#steps in the GD : Batch of the input / Pass it to the model / Compute loss function / Update the weights

def train(dataloader,model,loss_fn,optimizer):
  model.train() #putting the model in the training mode
  for batch,(X,y) in enumerate(dataloader):
    #sending the data to the GPU
    X = X.to(device)
    y = y.to(device)

    #Compute predictions
    pred = model(X)

    #Compute loss
    loss = loss_fn(pred,y)

    #Backpropogation
    loss.backward() #Wnew = Wold - lr*dl/dw
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print(f'Loss of the Model {loss.item()}')


In [13]:
def test(dataloader,model,loss_fn):
  model.eval() #putting the model in the training mode
  num_batched = len(dataloader)
  test_loss, correct = 0,0
  with torch.no_grad(): #We will not compute gradients for the test data
    for X,y in dataloader:
      X = X.to(device)
      y = y.to(device)

      #Compute predictions
      pred = model(X)

      #Compute loss
      test_loss += loss_fn(pred,y).item()

      #Find how many correct predictions
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss = test_loss/num_batched
  correct = correct/(len(dataloader.dataset))

  print(f'Test Accuracy {100*correct}, Avg_loss : {test_loss}')


In [14]:
epochs = 5

for t in range(5):
  print(f'Epoch {t+1}')
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)


Epoch 1
Loss of the Model 2.2981934547424316
Loss of the Model 2.2877535820007324
Loss of the Model 2.2648966312408447
Loss of the Model 2.264340877532959
Loss of the Model 2.2448341846466064
Loss of the Model 2.2199087142944336
Loss of the Model 2.2235374450683594
Loss of the Model 2.1929092407226562
Loss of the Model 2.1876072883605957
Loss of the Model 2.1571896076202393
Test Accuracy 54.22, Avg_loss : 2.1491740083998176
Epoch 2
Loss of the Model 2.1570186614990234
Loss of the Model 2.1439690589904785
Loss of the Model 2.0889885425567627
Loss of the Model 2.1115853786468506
Loss of the Model 2.0485575199127197
Loss of the Model 2.0040485858917236
Loss of the Model 2.0265016555786133
Loss of the Model 1.9525222778320312
Loss of the Model 1.9594354629516602
Loss of the Model 1.8843637704849243
Test Accuracy 58.919999999999995, Avg_loss : 1.8800443646254812
Epoch 3
Loss of the Model 1.9110862016677856
Loss of the Model 1.8773503303527832
Loss of the Model 1.7673988342285156
Loss of the

In [15]:
torch.save(model.state_dict(),"/content/iteration1.pth")

In [16]:

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/content/iteration1.pth"))

<All keys matched successfully>

In [17]:

classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle Boot"]

model.eval()
X,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  X = X.to(device)
  pred = model(X)
  predicted,actual = classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted {predicted}')
  print(f'Actual {actual}')

Predicted Ankle Boot
Actual Ankle Boot


In [18]:
y

9